In [1]:
import numpy as np, matplotlib.pyplot as plt
from collections import OrderedDict as od
from scipy.special import comb as C
from scipy.stats import linregress
from statistics import median
from scipy.optimize import fminbound
import json
import itertools
from tqdm.notebook import tqdm
#from os import walk
#import re

import mpmath as mp
mp.mp.dps = 50
mp.mp.pretty = True
print(mp.mp)

from math import sqrt
from joblib import Parallel, delayed

Mpmath settings:
  mp.prec = 169               [default: 53]
  mp.dps = 50                 [default: 15]
  mp.trap_complex = False     [default: False]


In [2]:
def get_entropy_from_culling_factor_v2(c):
    S = od()
    sn = 0
    for E in c:
        S[E] = np.log(c[E])
        S[E] += sn
        sn += np.log(1 - c[E])
    return S

def get_entropy_from_culling_factor_with_heat_v2(c):
    S = od()
    sn = 0
    for E in c:
        S[E] = np.log(c[E])
        S[E] += sn
        sn += np.log(1 - c[E])
    return S

In [11]:
def read_files_v2(parameters):
    q=parameters["q"];
    L=parameters["L"];
    R=parameters["R"];
    D=parameters["D"];
    nSteps=parameters["nSteps"];
    seed=parameters["seed"];
    heat=parameters["heat"];
    N = L * L;
    name = "./datasets/2DBlume{}_q{}_D{}_N{}_R{}_nSteps{}_run{}X.txt".format("Heating" * heat, q, "{0:07.6f}".format(D), N, R, nSteps, seed)
    #name = "./datasets/2DBlumeOld(FloatEnergy)/2DBlume{}_q{}_D{}_N{}_R{}_nSteps{}_run{}X.txt".format("Heating" * heat, q, "{0:07.6f}".format(D), N, R, nSteps, seed)
    #print(name)
    culling_factor = od()
    #print(N, R, nSteps, heat)
    with open(name) as f:
        for line in f:
            E, c = map(float, line.split())
            culling_factor[E] = c
    parameters["culling_factor"] = culling_factor
    if not heat:
        parameters["S"] = get_entropy_from_culling_factor_v2(culling_factor)
    else:
        parameters["S"] = get_entropy_from_culling_factor_with_heat_v2(culling_factor)
    
    return parameters

def read_tuple_files(tpl):
    if tpl[1]["heat"]:
        return {
            "cool": read_files_v2(tpl[0]),
            "heat": read_files_v2(tpl[1])
        }

def stitch_S(heat, cool):
    #not infinite values
    niv = sorted( list({E for E, S in heat["S"].items() if not np.isinf(S)} &
                       {E for E, S in cool["S"].items() if not np.isinf(S)}) )
    #chosen not infinite values
    cniv = niv[len(niv) // 3 : 2 * len(niv) // 3]
    #print(cniv)
    shift = 0
    count = 0
    for E in cniv:
        shift += heat["S"][E] - cool["S"][E]
        count += 1
    shift /= count

    result = od()
    for E in cool["S"]:
        if E < median(cniv):
            result[E] = cool["S"][E]
    for E in heat["S"]:
        if E >= median(cniv):
            result[E] = heat["S"][E] - shift
    return {"S": result, "L": heat["L"], "D": heat["D"], "R": heat["R"]}

def calc_hc(st, T):

    fe = mp.mpf(1.0) * np.array([e for e, S in st['S'].items()])
    fS = mp.mpf(1.0) * np.array([S for e, S in st['S'].items()])

    exp = np.frompyfunc(mp.exp, 1, 1)
    w = exp(fS - fe / T)

    Z = w.sum()
    E = (w * fe).sum()
    E_sq = (w * fe * fe).sum()
    E_qr = (w * fe * fe * fe * fe).sum()
    avgE = E / Z;
    avgE_sq = E_sq / Z;
    avgE_qr = E_qr / Z;
    # T, C, average E, BinderCumulant
    return T, float( (avgE_sq - avgE * avgE) / (T * T) ), float( avgE ), float(1 - (avgE_qr / (3 * avgE_sq * avgE_sq)))


In [12]:
T_crit = {}

In [13]:
D = 1.967
L = [x for x in [8, 12, 16, 20, 24, 32, 48, 64, 80, 96] if x >=0]

R = [32768, 131072]

cool, heat = [{
    "q": 3,
    "D": D,
    "L": l,
    #"R" : R,
    "nSteps" : 10,
    "seed" : 0,
    "heat": False
} for l in L], [{
    "q": 3,
    "D": D,
    "L": l,
    #"R" : R,
    "nSteps" : 10,
    "seed" : 0,
    "heat": True
} for l in L]
print('D={}'.format(D))

parameters = []
for c, h in zip(cool, heat):
    try:
        c['R'] = R[1]
        h['R'] = R[1]
        parameters.append(read_tuple_files((c, h)))
    except:
        pass
#        try:
#            c['R'] = R[1]
#            h['R'] = R[1]
#            parameters.append(read_tuple_files((c, h)))
#        except:
#            pass
        
for i, x in enumerate(tqdm(parameters)):
    parameters[i]["stitched"] = stitch_S(x["heat"], x["cool"])
print('L in {}'.format(", ".join([str(x["stitched"]["L"]) for x in parameters]) ))
print('R in {}'.format(", ".join([str(x["heat"]["R"]) for x in parameters]) ))


D=1.967


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()



L in 8, 12, 16, 20, 24, 32
R in 131072, 131072, 131072, 131072, 131072, 131072


In [ ]:
for p in parameters:
    cool, heat, st = p['cool'], p['heat'], p['stitched']

    plt.figure(figsize=(16, 9))

    plt.plot([x for x in cool['culling_factor']], [y for x, y in cool['culling_factor'].items()], 'o')
    plt.plot([x for x in heat['culling_factor']], [y for x, y in heat['culling_factor'].items()], 'o')
    plt.grid()

    plt.title("$D={}; L={}; R={}$".format(st["D"], st["L"], st["R"]), fontsize=16)
    plt.ylabel('$culling\_factor(E)$', fontsize=16)
    plt.xlabel('$E$', fontsize=16)
    plt.show()
    #plt.savefig("./images/BC(T)_D{}_L{}.png".format(st["D"], st["L"]))


In [ ]:
for p in parameters:

    plt.figure(figsize=(16, 9))

    cool, heat, st = p['cool'], p['heat'], p['stitched']

    plt.plot([x for x in st['S']], [y for x, y in st['S'].items()], 'o')
    plt.grid()
    plt.title("$D={}; L={}$".format(st["D"], st["L"], st["R"]), fontsize=16)
    plt.ylabel('$S(E)$', fontsize=16)
    plt.xlabel('$E$', fontsize=16)
    #savefig("./images/S(E)_D{}_L{}.png".format(st["D"], st["L"]))
    plt.show()

In [ ]:
D

In [ ]:
hc = {}
E = {}
BC = {}

In [ ]:
for i, x in enumerate(tqdm(parameters)):
    heat, cool, st = parameters[i]["heat"], parameters[i]["cool"], parameters[i]["stitched"]
    L = st['L']
    N = L * L

    hc[L] = {}
    E[L] = {}
    BC[L] = {}


    T_list = np.linspace(0.2, 0.75, 20)
    

    # T, C, average E, BinderCumulant
    for z in Parallel(n_jobs=4)(delayed(calc_hc)(st, T) for T in tqdm(T_list, desc='L={}'.format(L))):
        t, a, b, c = z
        hc[L][t], E[L][t], BC[L][t] = a, b, c

    #for T in tqdm(T_list):
    #    T, hc[L][T], E[L][T], BC[L][T] = calc_hc(st, T)


In [ ]:
#BC pictures
for i, x in enumerate(tqdm(parameters)):
    plt.figure(figsize=(16, 9))
    heat, cool, st = parameters[i]["heat"], parameters[i]["cool"], parameters[i]["stitched"]
    L = st['L']
    N = L * L
    

    plt.plot([T for T in BC[L] if T >= 0.0], [c / N for T, c in BC[L].items() if T >= 0.0], 'o')
    plt.grid()
    plt.title("$D={}; L={}$".format(st["D"], st["L"]), fontsize=16)
    plt.ylabel('$BC$', fontsize=16)
    plt.xlabel('$T$', fontsize=16)
    plt.show()
    #plt.savefig("./images/BC(T)_D{}_L{}.png".format(st["D"], st["L"]))


In [ ]:
plt.figure(figsize=(16, 9))
for i, x in enumerate(tqdm(parameters)):
    heat, cool, st = parameters[i]["heat"], parameters[i]["cool"], parameters[i]["stitched"]
    L = st['L']
    N = L * L
    plt.plot([T for T in BC[L]], [c for T, c in BC[L].items()], 'o', label="L={}".format(L))
plt.grid()
plt.title("$D={}$".format(st["D"]), fontsize=16)
plt.ylabel('$BC$', fontsize=16)
plt.xlabel('$T$', fontsize=16)
plt.legend()
#plt.show()
plt.savefig("./images/BC(T)_D{}.png".format(st["D"]))

In [ ]:
#E_avg pictures
for i, x in enumerate(tqdm(parameters)):
    plt.figure(figsize=(16, 9))
    heat, cool, st = parameters[i]["heat"], parameters[i]["cool"], parameters[i]["stitched"]
    L = st['L']
    N = L * L


    plt.plot([T for T in E[L]], [e / N for T, e in E[L].items()], 'o')
    plt.grid()
    plt.title("$D={}; L={}$".format(st["D"], st["L"]), fontsize=16)
    plt.ylabel('$E_{avg}$', fontsize=16)
    plt.xlabel('$T$', fontsize=16)
    #plt.show()
    plt.savefig("./images/E_avg(T)_D{}_L{}.png".format(st["D"], st["L"]))


In [ ]:
plt.figure(figsize=(16, 9))
for i, x in enumerate(tqdm(parameters)):
    heat, cool, st = parameters[i]["heat"], parameters[i]["cool"], parameters[i]["stitched"]
    L = st['L']
    N = L * L
    plt.plot([T for T in E[L]], [e / N for T, e in E[L].items()], 'o')
plt.grid()
plt.title("$D={}$".format(st["D"]), fontsize=16)
plt.ylabel('$E_{avg}$', fontsize=16)
plt.xlabel('$T$', fontsize=16)
#plt.show()
plt.savefig("./images/E_avg(T)_D{}.png".format(st["D"]))


In [ ]:
#C pictures
for i, x in enumerate(tqdm(parameters)):
    plt.figure(figsize=(16, 9))
    heat, cool, st = parameters[i]["heat"], parameters[i]["cool"], parameters[i]["stitched"]
    L = st['L']
    N = L * L


    plt.plot([T for T in hc[L]], [c / N for T, c in hc[L].items()], 'o')
    plt.grid()
    plt.title("$D={}; L={}$".format(st["D"], st["L"]), fontsize=16)
    plt.ylabel('$C$', fontsize=16)
    plt.xlabel('$T$', fontsize=16)
    #plt.show()
    plt.savefig("./images/C(T)_D{}_L{}.png".format(st["D"], st["L"]))


In [ ]:
plt.figure(figsize=(16, 9))
for i, x in enumerate(tqdm(parameters)):
    heat, cool, st = parameters[i]["heat"], parameters[i]["cool"], parameters[i]["stitched"]
    L = st['L']
    N = L * L
    plt.plot([T for T in hc[L]], [c / N for T, c in hc[L].items()], 'o')
plt.grid()
plt.title("$D={}; L={}$".format(st["D"], st["L"]), fontsize=16)
plt.ylabel('$C$', fontsize=16)
plt.xlabel('$T$', fontsize=16)
#plt.show()
plt.savefig("./images/C(T)_D{}.png".format(st["D"]))


In [ ]:
D

In [ ]:
parameters

In [7]:
BC_crit = {} #{"T1": , "BC1": }

In [14]:
for p in tqdm(parameters):
    st = p["stitched"]
    L = st["L"]
    print(L)
    BC_crit[L] = {}
    a = 0.4
    b = 0.6
    #T_list = np.linspace(0.3, 4, 100)
    #for T in tqdm(T_list):
        # T, C, average E, BinderCumulant
    t, bc, _, _ = fminbound(lambda T: calc_hc(st, T)[3], a, b, maxfun=500, full_output=1, disp=False)
        #t, c, _, bc = calc_hc(st, T)
    BC_crit[L]["T"] = float(t)
    BC_crit[L]["BC"]= float(bc)
    print(BC_crit[L])

8
{'T': 0.47485371979759544, 'BC': -0.9426490864517812}
12
{'T': 0.47688548073952525, 'BC': -0.4649564994494558}
16
{'T': 0.4711049114112405, 'BC': -0.30851703496315996}
20
{'T': 0.46707517187092096, 'BC': -0.23399567493226783}
24
{'T': 0.4651414169066059, 'BC': -0.17843610605671217}
32
{'T': 0.46177366609592957, 'BC': -0.13363554223514634}



In [17]:
D

1.967

In [19]:
with open("./cluster_tmp/BC_crit_D{}.txt".format(D), 'w') as fp:
    json.dump(BC_crit, fp)
    #for L in T_crit:
    #    fp.write('{}\t{}\t{}\n'.format(L, T_crit[L]['T_crit'], T_crit[L]['BC_crit']))

with open('./cluster_tmp/BC_crit_D{}.tsv'.format(D), 'w') as f:
    f.write('L\tT_crit\tBC_crit\n');
    for L in BC_crit:
        f.write('\t'.join([str(L), str(BC_crit[L]['T']), str(BC_crit[L]['BC'])]) + '\n')


In [ ]:
#with open("./cluster_tmp/BC_crit_D{}.txt".format(D), 'r') as fp:
#    BC_crit = json.load(fp)


In [ ]:
BC_crit

In [ ]:
for D in [1.0, 1.5, 1.6, 1.7, 1.75, 1.87, 1.9, 1.92, 1.94, 1.95, 1.96, 1.962, 1.964, 1.965, 1.965, 1.966, 1.967, 1.97, 1.98, 1.99]:
    try:
        with open('./cluster_tmp/BC_crit_D{}.txt'.format(D), 'r') as f:
            BC_crit = json.load(f)

        with open('./cluster_tmp/BC_crit_D{}.tsv'.format(D), 'w') as f:
            f.write('L\tT_crit\tBC_crit\n');
            for L in BC_crit:
                f.write('\t'.join([L, str(BC_crit[L]['T']), str(BC_crit[L]['BC'])]) + '\n')
    except:
        pass

In [ ]:
BC_crit, BC_crit_second_peak = {}, {}

In [ ]:
for D in [1.964, 1.965, 1.966, 1.968, 1.969]:#1.967, 
    with open('./old_tmp/BC_crit_D{}.txt'.format(D), 'r') as f:
        for line in f:
            arr = line[:-1].split('\t')
            L = int(arr[0])
            BC_crit[L] = {'T': float(arr[1]), 'BC': float(arr[2])}
            if len(arr) > 3:
                BC_crit_second_peak[L] = {'T': float(arr[3]), 'BC': float(arr[4])}
        
        with open('./old_tmp/BC_crit_D{}.tsv'.format(D), 'w') as f:
            f.write('L\tT_crit\tBC_crit\n');
            for L in BC_crit:
                f.write('\t'.join([str(L), str(BC_crit[L]['T']), str(BC_crit[L]['BC'])]) + '\n')
        
        with open('./old_tmp/BC_crit_D{}_second_peak.tsv'.format(D), 'w') as f:
            f.write('L\tT_crit\tBC_crit\n');
            for L in BC_crit_second_peak:
                f.write('\t'.join([str(L), str(BC_crit_second_peak[L]['T']), str(BC_crit_second_peak[L]['BC'])]) + '\n')
